In [1]:
import yaml
import itertools
import re
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

from src.config import read_yaml_config, get_value_from_config
from src.io import read_fasta_as_dict, get_augustus_proteins, read_predictions
from src.visual import visualize_predictions, visualize_single_prediction


warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

In [2]:
config = read_yaml_config("config.yaml")

In [3]:
TARGET_SPECIES = "Andrena_fulva"

In [4]:
TARGET_SPECIES = TARGET_SPECIES.replace(" ", "_")
genome_file = Path(config['data_locations']['genomes_dir']) / f'{TARGET_SPECIES}.fna'
proteome_file = Path(config['data_locations']['proteomes_dir']) / f'{TARGET_SPECIES}.faa'
predictions_file = Path(config['data_locations']['raw_predictions_dir']) / f'{TARGET_SPECIES}.tsv'
genes_file = Path(config['data_locations']['gene_predictions_dir']) / f'{TARGET_SPECIES}.fna'
annotation_file = Path(config['data_locations']['gene_annotations_dir']) / f'{TARGET_SPECIES}.gff'
false_positives_file = Path(config['data_locations']['false_positives_dir']) / f'{TARGET_SPECIES}.faa'

In [6]:
genome_map = read_fasta_as_dict(genome_file)
proteome_map = read_fasta_as_dict(proteome_file)
# genes_map = read_fasta_as_dict(genes_file)
# fp_map = read_fasta_as_dict(false_positives_file)
predictions_data = read_predictions(predictions_file)

In [7]:
predictions_data.head(15)

,pos_count,model_name,record_description,prediction_mask,sequence,g_record_id,t_record_id,frame
0,153,HybridModel_1000_cuda_v1,OX276327.1_start=39565000_end=39595000_frame=-...,0000000000000000000000000000000000000000000000...,LCHTVTKSNAVIVTFTFHISYSFVISF**NICGPMFM*Y*HVIFNH...,OX276327.1_start=39565000_end=39595000_frame=-,OX276327.1_start=39565000_end=39595000_frame=-2,2
1,152,HybridModel_1000_cuda_v1,OX276327.1_start=39565000_end=39595000_frame=-...,0000000000000000000000000000000000000000000000...,FMSHCY*K*CRYRYLYISYFI*FCYKFLIKHLRTYVYVILTCYF*P...,OX276327.1_start=39565000_end=39595000_frame=-,OX276327.1_start=39565000_end=39595000_frame=-1,1
2,152,HybridModel_1000_cuda_v1,OX276327.1_start=39565000_end=39595000_frame=-...,0000000000000000000000000000000000000000000000...,YVTLLLKVMPLSLPLHFIFHIVLL*VFNKTFADLCLCDINMLFLTT...,OX276327.1_start=39565000_end=39595000_frame=-,OX276327.1_start=39565000_end=39595000_frame=-3,3
3,96,HybridModel_1000_cuda_v1,OX276327.1_start=2995000_end=3025000_frame=+3 ...,0000000000000000000000000000000000000000000000...,ESRG*FRRRGAKRGPAVPRERAID*RRTKSNKHIILSSR*LNATYF...,OX276327.1_start=2995000_end=3025000_frame=+,OX276327.1_start=2995000_end=3025000_frame=+3,3
4,94,HybridModel_1000_cuda_v1,OX276327.1_start=2995000_end=3025000_frame=+1 ...,0000000000000000000000000000000000000000000000...,ERAAGDSGVEEPSVARQFHESERSTSDERNRINILFYLVGNSMPLI...,OX276327.1_start=2995000_end=3025000_frame=+,OX276327.1_start=2995000_end=3025000_frame=+1,1
5,86,HybridModel_1000_cuda_v1,OX276327.1_start=3025000_end=3055000_frame=+2 ...,0000000000000000000000000000000000000000000000...,IYK**D*LAKKYP*TQTQTRFITYNRIELVVERRYWKAPIRFAFQN...,OX276327.1_start=3025000_end=3055000_frame=+,OX276327.1_start=3025000_end=3055000_frame=+2,2
6,81,HybridModel_1000_cuda_v1,OX276327.1_start=2995000_end=3025000_frame=+2 ...,0000000000000000000000000000000000000000000000...,REPRVIPASRSQAWPGSSTRASDRLATNEIE*TYYFI**VTQCHLF...,OX276327.1_start=2995000_end=3025000_frame=+,OX276327.1_start=2995000_end=3025000_frame=+2,2
7,69,HybridModel_1000_cuda_v1,OX276327.1_start=3025000_end=3055000_frame=+1 ...,0000000000000000000000000000000000000000000000...,YLQIIRLTREKISVDANANTIYHIQSD*IGS*TKILESTDKIRVSE...,OX276327.1_start=3025000_end=3055000_frame=+,OX276327.1_start=3025000_end=3055000_frame=+1,1
8,65,HybridModel_1000_cuda_v1,OX276327.1_start=3025000_end=3055000_frame=+3 ...,0000000000000000000000000000000000000000000000...,FTNNKTDSRKNIRRRKRKHDLSHTIGLNW*LNEDIGKHR*DSRFRT...,OX276327.1_start=3025000_end=3055000_frame=+,OX276327.1_start=3025000_end=3055000_frame=+3,3
9,9,HybridModel_1000_cuda_v1,OX276328.1_start=26725000_end=26755000_frame=+...,0000000000000000000000000000000000000000000000...,AGCDGRVPSH*KDGAPT*RESL*WCIPITYTTMTCCRPSP*RCEIV...,OX276328.1_start=26725000_end=26755000_frame=+,OX276328.1_start=26725000_end=26755000_frame=+3,3


In [8]:
SCORE_THRESHOLD = 30

selected_data = predictions_data.query("pos_count > @SCORE_THRESHOLD")

# Visualize predictions

In [9]:
SHOW_NUCLEOTIDE = True
SHOW_TRANSLATED = True
SHOW_MISSING_FRAMES = False
NUCL_VISUAL_OFFSET = np.array((1500, 500))
AA_VISUAL_OFFSET = NUCL_VISUAL_OFFSET // 3


visualize_predictions(
    selected_data,
    genome_map=genome_map,
    proteome_map=proteome_map,
    show_translated=SHOW_TRANSLATED,
    show_nucleotide=SHOW_NUCLEOTIDE,
    show_missing_frames=SHOW_MISSING_FRAMES,
    nucl_offset=NUCL_VISUAL_OFFSET,
    aa_offset=AA_VISUAL_OFFSET
)

OX276327.1_start=39565000_end=39595000_frame=-2 Score=153 FragmentStart=0_FragmentEnd=818
SKKSFFHSTVLFILF*YEIDIVHILC*ITSIRKGYFRIQKMMIVPQN*YHVCYAMRKQQMFGQRVGITRGDTPSGIFEVLRSLVPLNWLY*AAVYLRLTIASC*SLLSPTTEWTFKLVSL*NCTLWFGIVTSSGGFKKKGPGGERFGGPTFSEKMRR*FTKRG*VYRKV*KKNIVVG*NHNFYF*RQGPRGPSEQGPFFRENRAVFY*KRAQ*NNTEKNIVWIKS*FFLRMGPWRGPLDRGPGGNYSSAALLNPPLVTSIIFSEHMYRIISLRKVSVLLSERSCFPTIQYIIRIQEVLLSTSVKSRHNGSKS*FRKKKPNVTEYFHVESRNSDMNKYFSVLVISSVQY*HFSL*KTKFLSKFLILIQCKIPLSSVINVT*ISSR*LTSVSSELKKSKLKVMNILVRQALAGAADAGTIWAQWTRLLTLLNTTYQYHLYFKVDCNLHSVYPRWTTRYLLK*TSLHLSNTPLNPATMPPNATIEIKKPFSYTATESLRPSHCDRVTATESLRPSHCDRVTATESLRPSHCDRVTATESLRPSHCDRVTATESLRPSHCDRVTATESLRPSHCDRVTATESLRPSHCDRVTATESLRPSHCDRVTATESLRPSHCDRVTATESLRPSHCDRVTATESLRPSHCDRVTATESLRPSHCDRVTATKSHTFSIHPSEKDALNPIAKPLKNPHLGQPLAKFEIEIVYSTKILFFIFTYVGAYLHSVYKLKS**TISTGNTMKACSENRTKEHSVTGENWHSCSTRRAPSNDQPVRKTYPLT*C*PNTGSCARGGTNSSLWFRRTPPRSSKSSQ*SLRK*LDPVLR
OX276327.1_start=39565000_end=39595000_frame=-1 Score=152 FragmentStart=0_FragmentEnd=821
I

# False Positives

## Overview

In [28]:
from src.io import FragmentRecordId
from src.predictions import extract_fp_aa_predictions

In [34]:
fp_rec_ids = {FragmentRecordId(rec_id) for rec_id in fp_map}
fp_predictions = extract_fp_aa_predictions(predictions_data.query("record_id in @fp_rec_ids"), kernel_size=33)
for _, row in fp_predictions.iterrows():
    visualize_single_prediction(row.sequence, row.prediction_mask, caption=f"{row.record_id} Score={row.pos_count}")
    print()

RJVV01491319.1_start=0_end=30000_frame=-1 Score=14 Fragment_start=0_end=33
TIG*NIPIFNYIILNFQRVRREPGRPGNMPRPK

RJVV01518257.1_start=0_end=30000_frame=-2 Score=14 Fragment_start=0_end=33
TIG*NIPIFNYIILNFQRVRREPGRPGNMPRPK

RJVV01002766.1_start=0_end=30000_frame=+2 Score=10 Fragment_start=0_end=43
SLHSASLNPFTLTDFLTFFHFMAPRTYRNFRFSQIRHPGVKSR

RJVV01011221.1_start=0_end=30000_frame=-1 Score=10 Fragment_start=0_end=43
QELRKMNTTRRFAVLHPFFKFIPPRWLHQSQIRTFHDKTVTLI

RJVV01040401.1_start=0_end=30000_frame=+2 Score=8 Fragment_start=0_end=40
ELTLDSEDLKRIFLGFPKNSQEYSREFHDFP*KSHKWPRI

RJVV01502259.1_start=0_end=30000_frame=+1 Score=8 Fragment_start=0_end=25
PIRPRPPHPVSTIFIENE**LIEKK

RJVV01507231.1_start=0_end=30000_frame=-1 Score=7 Fragment_start=0_end=23
NLPGEESDDESIDEGYDSPLDRT

RJVV01380973.1_start=0_end=30000_frame=+3 Score=6 Fragment_start=0_end=41
TIRNVNKKFF*LLLFCFVPRASTKNYREIHHGKFKLLRNKY

RJVV01008589.1_start=0_end=30000_frame=-1 Score=5 Fragment_start=0_end=38
RTLRRVISPVLRRDFTTFFPRAQERDSHGTYRF

## Detailed

In [12]:
TARGET_RECORD_ID = "RJVV01491319.1_start=0_end=30000_frame=-"
SHOW_NUCLEOTIDE = True
SHOW_TRANSLATED = True
SHOW_MISSING_FRAMES = True
NUCL_VISUAL_OFFSET = np.array((1500, 500))
AA_VISUAL_OFFSET = NUCL_VISUAL_OFFSET // 3


fp_selected_data = predictions_data[predictions_data.record_id.astype(str).str.startswith(TARGET_RECORD_ID)]


visualize_predictions(fp_selected_data, genomic_map, proteome_map, show_translated=SHOW_TRANSLATED, show_nucleotide=SHOW_NUCLEOTIDE, show_missing_frames=SHOW_MISSING_FRAMES, nucl_visual_offset=NUCL_VISUAL_OFFSET)

RJVV01491319.1_start=0_end=30000_frame=-1 Score=14 Fragment_start=0_end=89
S*RT*SRRP*SR*RFALRGNKRSIFYCVKI*L*KHFI*EN*NLF*N*NKNKIF*KGTIG*NIPIFNYIILNFQRVRREPGRPGNMPRPK
RJVV01491319.1_start=0_end=30000_frame=-2 Score=0 Fragment_start=0_end=88
AEELEVEDLEADDVLPLEVTKDRFFIV*KYNYKNILFKKIEIYFKIKTKIKYFKKGQLAKIFQFLII*F*TFREFVENQAVQEICHDL
RJVV01491319.1_start=0_end=30000_frame=-3 Score=0 Fragment_start=0_end=88
LKNLK*KTLKQMTFCP*R*QKIDFLLCENIIIKTFYLRKLKFILKLKQK*NILKRDNWLKYSNF*LYNFKLSESSSRTRPSRKYATT*
RJVV01491319.1_start=0_end=30000_frame=- Fragment_start=0_end=267
AGCTGAAGAACTTGAAGTAGAAGACCTTGAAGCAGATGACGTTTTGCCCTTAGAGGTAACAAAAGATCGATTTTTTATTGtgtgaaaatataattataaaaacattttatttaagaaaattgaaatttattttaaaattaaaacaaaaataaaatattttaaaaagggaCAATTGGctaaaatattccaatttttaattatataattttaaactttcagagAGTTCGTCGAGAACCAGGCCGTCCAGGAAATATGCCACGACCTAAA
                                                                                                                        


In [35]:
TARGET_RECORD_ID = "RJVV01491319.1_start=0_end=30000_frame=-"
SHOW_NUCLEOTIDE = True
SHOW_TRANSLATED = True
SHOW_MISSING_FRAMES = True
NUCL_VISUAL_OFFSET = np.array((1500, 500))
AA_VISUAL_OFFSET = NUCL_VISUAL_OFFSET // 3


fp_selected_data = predictions_data[predictions_data.record_id.astype(str).str.startswith(TARGET_RECORD_ID)]


visualize_predictions(fp_selected_data, genome_map=genome_map, proteome_map=proteome_map, show_translated=SHOW_TRANSLATED, show_nucleotide=SHOW_NUCLEOTIDE, show_missing_frames=SHOW_MISSING_FRAMES, nucl_offset=NUCL_VISUAL_OFFSET)

RJVV01491319.1_start=0_end=30000_frame=-1 Score=14 Fragment_start=0_end=89
S*RT*SRRP*SR*RFALRGNKRSIFYCVKI*L*KHFI*EN*NLF*N*NKNKIF*KGTIG*NIPIFNYIILNFQRVRREPGRPGNMPRPK
RJVV01491319.1_start=0_end=30000_frame=-2 Score=0 Fragment_start=0_end=88
AEELEVEDLEADDVLPLEVTKDRFFIV*KYNYKNILFKKIEIYFKIKTKIKYFKKGQLAKIFQFLII*F*TFREFVENQAVQEICHDL
RJVV01491319.1_start=0_end=30000_frame=-3 Score=0 Fragment_start=0_end=88
LKNLK*KTLKQMTFCP*R*QKIDFLLCENIIIKTFYLRKLKFILKLKQK*NILKRDNWLKYSNF*LYNFKLSESSSRTRPSRKYATT*
RJVV01491319.1_start=0_end=30000_frame=- Fragment_start=0_end=267
AGCTGAAGAACTTGAAGTAGAAGACCTTGAAGCAGATGACGTTTTGCCCTTAGAGGTAACAAAAGATCGATTTTTTATTGtgtgaaaatataattataaaaacattttatttaagaaaattgaaatttattttaaaattaaaacaaaaataaaatattttaaaaagggaCAATTGGctaaaatattccaatttttaattatataattttaaactttcagagAGTTCGTCGAGAACCAGGCCGTCCAGGAAATATGCCACGACCTAAA
                                                                                                                        


# Proteins

In [14]:
records = get_augustus_proteins(annotation_file)
for idx, record in enumerate(records):
    print(idx, record.id, record.description)
    print(record.seq)
    print()

0 RJVV01031371.1_start=0_end=30000_frame=+ Gene_start=1393 Gene_end=3525 Auto
MAKISAFLIVALFAFAALSVQAEPEPARGGKPSRPRPPPIKPRPPHPRLRREAEGLEEDVAEVETDEVEESAVALDRVRREPGRPGNMPRPKPIPIRPRPPHPRLRREAEELEAEDVLPLERLRREAEELEAEDLEADEVLPLERVRREPGRPGNMPRPKPIPIRPRPPHPRLRREAEELEVEDLEADDVLPLERVRREPGRPGNMPRPKPIPIRPRPPHPVSIIFIENE

1 RJVV01183868.1_start=0_end=30000_frame=- Gene_start=28 Gene_end=535 Auto
MPRPKPIPIRPRPPHPRLRREAEELEVEDLEADDVLPLERVRREPGRPGNMPRPKPIPIRPRPPHPVSTIFIENE

2 RJVV01026129.1_start=0_end=30000_frame=+ Gene_start=1395 Gene_end=2603 Auto
MAKISAFLIVALFAFAALSVQAEPEPARGGKPSRPRPPPIKPRPPHPRLRREAEGLEEDVAEVETDEVEESAVALDRVRREPGRPGNMPRPKPIPIRPRPPHPRLRREAEELEAEDVLPLESPSRTRPSRKYATT



# Commit proteins

In [21]:
def commit_proteins(proteins_file, new_records, force=False):
    all_proteins = {rec.id: rec for rec in SeqIO.parse(proteins_file, "fasta")}
    for record in new_records:
        if record.id in all_proteins and not force:
            warnings.warn(f"Protein {record.id} is already commited. Skipping...", category=UserWarning)
            continue
        all_proteins[record.id] = record
    return SeqIO.write(list(all_proteins.values()), proteins_file, "fasta")

## Commit auto

In [22]:
subset_to_commit = [0, 1, 2]
records_to_commit = [records[idx] for idx in subset_to_commit]
for idx, record in enumerate(records_to_commit):
    print(idx, record.id, record.description)
    print(record.seq)
    print()

0 RJVV01031371.1_start=0_end=30000_frame=+ Gene_start=1393 Gene_end=3525 Auto
MAKISAFLIVALFAFAALSVQAEPEPARGGKPSRPRPPPIKPRPPHPRLRREAEGLEEDVAEVETDEVEESAVALDRVRREPGRPGNMPRPKPIPIRPRPPHPRLRREAEELEAEDVLPLERLRREAEELEAEDLEADEVLPLERVRREPGRPGNMPRPKPIPIRPRPPHPRLRREAEELEVEDLEADDVLPLERVRREPGRPGNMPRPKPIPIRPRPPHPVSIIFIENE

1 RJVV01183868.1_start=0_end=30000_frame=- Gene_start=28 Gene_end=535 Auto
MPRPKPIPIRPRPPHPRLRREAEELEVEDLEADDVLPLERVRREPGRPGNMPRPKPIPIRPRPPHPVSTIFIENE

2 RJVV01026129.1_start=0_end=30000_frame=+ Gene_start=1395 Gene_end=2603 Auto
MAKISAFLIVALFAFAALSVQAEPEPARGGKPSRPRPPPIKPRPPHPRLRREAEGLEEDVAEVETDEVEESAVALDRVRREPGRPGNMPRPKPIPIRPRPPHPRLRREAEELEAEDVLPLESPSRTRPSRKYATT



In [23]:
commit_proteins("hymenoptera_proteins.faa", records_to_commit)

3

## Commit custom

In [168]:
custom_record = SeqRecord(
    Seq(""),
    id="",
    description=f"Gene_start={} Gene_end={} Manual"
)
custom_record

SyntaxError: f-string: empty expression not allowed (1575694441.py, line 5)

In [ ]:
# commit_proteins("hymenoptera_proteins.faa", [custom_record])